En este **notebook** se realiza el preprocesamiento y la preparación del dataset seleccionado para entrenar un modelo de clasificación emocional de tweets en español. El dataset proviene de Kaggle y contiene ejemplos etiquetados con distintas emociones.

1) **Carga del dataset**

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Cargar el archivo CSV respetando las comillas
df = pd.read_csv("/content/drive/MyDrive/Deep Learnig/Integrador/Segundo Entregable/sentiment_analysis_dataset.csv",
                 delimiter=',',)
# Ver columnas
print("Columas del DataSet: ", df.columns.tolist())

# Seleccionar solo 'text' y 'emotion'
df = df[['text', 'emotion']]
#Forma del dataset (Filas,Columnas)
print("Forma limpia del dataset [Filas,Columnas]:", df.shape)
#Mostramos los primeros datos
print("Los primeros 5 datos del DataSet: ")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columas del DataSet:  ['user', 'text', 'date', 'emotion', 'sentiment']
Forma limpia del dataset [Filas,Columnas]: (2590, 2)
Los primeros 5 datos del DataSet: 


,text,emotion
0,termine bien abrumado después de hoy,overwhelmed
1,me siento abrumado,overwhelmed
2,Me siento un poco abrumado por la cantidad de ...,overwhelmed
3,Salvador la única persona que no la ha abrumad...,overwhelmed
4,Denme un helado o algo que ando full abrumado.,overwhelmed


**Traducimos los datos al español...**

In [ ]:

translation = {
    'joyful': 'Felicidad',
    'daring': 'Atrevimiento',
    'optimistic': 'Optimista',
    'playful': 'Jugueton',

    'powerful': 'Empoderado',
    'surprised': 'Sorpresa',
    'successful': 'Triunfo',
    'confident': 'Confiado',

    'peaceful': 'Paz',
    'secure': 'Seguro',
    'thankful': 'Agradecido',
    'loving': 'Enamorado',
    'relaxed': 'Relajado',
    'responsive': 'Responsable',

    'sad': 'Tristeza',
    'sleepy': 'Adormilado',
    'isolated': 'Aislado',
    'stupid': 'Tomto ;B',

    'mad': 'Furia',
    'distant': 'Distante',
    'frustrated': 'Frustrado',
    'irritated': 'Irritado',
    'jealous': 'Celoso',

    'scared': 'Miedo',
    'embarrassed': 'Apenado',
    'overwhelmed': 'Preocupado',

}

In [ ]:
df[['emotion']] = df[['emotion',]].replace(to_replace = translation)
df

,text,emotion
0,termine bien abrumado después de hoy,Preocupado
1,me siento abrumado,Preocupado
2,Me siento un poco abrumado por la cantidad de ...,Preocupado
3,Salvador la única persona que no la ha abrumad...,Preocupado
4,Denme un helado o algo que ando full abrumado.,Preocupado
...,...,...
2585,No podemos vivir con miedo: ¡Manejen borrach...,Atrevimiento
2586,"La vida es un constante, SIN MIEDO AL ÉXITO 💅🏼",Atrevimiento
2587,Esquizofrenia = mente dividida: Miedo a las re...,Atrevimiento
2588,"""Lo que más miedo me da, es ver cómo desaparec...",Atrevimiento


No charts were generated by quickchart


2)**Expolarion de datos**

In [ ]:
# Ver distribución de etiquetas
df["emotion"].value_counts()

,count
emotion,
Agradecido,150
Adormilado,150
Preocupado,140
Celoso,140
Triunfo,140
Irritado,140
Seguro,140
Confiado,140
Sorpresa,140


3) **Preprocesamiento de los datos**


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)       # eliminar URLs
    text = re.sub(r"@\w+", "", text)          # eliminar menciones
    text = re.sub(r"#\w+", "", text)          # eliminar hashtags
    text = re.sub(r"[^\w\s]", "", text)       # eliminar signos de puntuación
    return text.strip()

df["texto_limpio"] = df["text"].apply(clean_text)
df[["text", "texto_limpio"]]

,text,texto_limpio
0,termine bien abrumado después de hoy,termine bien abrumado después de hoy
1,me siento abrumado,me siento abrumado
2,Me siento un poco abrumado por la cantidad de ...,me siento un poco abrumado por la cantidad de ...
3,Salvador la única persona que no la ha abrumad...,salvador la única persona que no la ha abrumad...
4,Denme un helado o algo que ando full abrumado.,denme un helado o algo que ando full abrumado
...,...,...
2585,No podemos vivir con miedo: ¡Manejen borrach...,no podemos vivir con miedo manejen borrachos...
2586,"La vida es un constante, SIN MIEDO AL ÉXITO 💅🏼",la vida es un constante sin miedo al éxito
2587,Esquizofrenia = mente dividida: Miedo a las re...,esquizofrenia mente dividida miedo a las real...
2588,"""Lo que más miedo me da, es ver cómo desaparec...",lo que más miedo me da es ver cómo desapareces...


4) **Codificación de etiquetas**

Convertimos las etiquetas de texto (emociones) en números, lo cual es necesario para entrenar nuestra red neuronal

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df["label_codificada"] = encoder.fit_transform(df["emotion"])

# Guardar el mapeo de etiquetas
label_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("Etiquetas codificadas:", label_mapping)

Etiquetas codificadas: {'Adormilado': np.int64(0), 'Agradecido': np.int64(1), 'Aislado': np.int64(2), 'Apenado': np.int64(3), 'Atrevimiento': np.int64(4), 'Celoso': np.int64(5), 'Confiado': np.int64(6), 'Distante': np.int64(7), 'Enamorado': np.int64(8), 'Frustrado': np.int64(9), 'Irritado': np.int64(10), 'Jugueton': np.int64(11), 'Optimista': np.int64(12), 'Preocupado': np.int64(13), 'Relajado': np.int64(14), 'Responsable': np.int64(15), 'Seguro': np.int64(16), 'Sorpresa': np.int64(17), 'Tomto ;B': np.int64(18), 'Triunfo': np.int64(19)}


5) **Tokenización con DistilBERT**

1. Toma los textos ya limpios.

2. Los convierte a tokens numéricos que entiende DistilBERT.

3. Aplica padding y truncado.

4. Devuelve tensores listos para usar en PyTorch.

5. El 0 al final es padding, para que todos los textos tengan la misma longitud.

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

tokens = tokenizer(
    df["texto_limpio"].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

6) **Definición del Dataset en PyTorch**

In [ ]:
from torch.utils.data import Dataset
import torch

class TweetEmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

labels = torch.tensor(df["label_codificada"].values)
dataset = TweetEmotionDataset(tokens, labels)

7) **División en train/test**

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["texto_limpio"].tolist(),
    df["label_codificada"].tolist(),
    test_size=0.2,
    random_state=42
)

# Se puede tokenizar de nuevo por separado si se desea:
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

train_dataset = TweetEmotionDataset(train_encodings, torch.tensor(train_labels))
val_dataset = TweetEmotionDataset(val_encodings, torch.tensor(val_labels))

8) **Guardar dataset preprocesado**

In [ ]:
# Guardar como CSV para referencias posteriores
df.to_csv("/content/drive/MyDrive/Deep Learnig/Integrador/Segundo Entregable/dataset_preprocesado.csv", index=False)